## GPT2 fine tuning

In [8]:
# Install dependencies
!pip install transformers pandas numpy torch -qq

ERROR: THESE PACKAGES DO NOT MATCH THE HASHES FROM THE REQUIREMENTS FILE. If you have updated the package versions, please update the hashes. Otherwise, examine the package contents carefully; someone may have tampered with them.
    pandas from https://files.pythonhosted.org/packages/cb/69/d2a9673f2fbb96f072c28f3bf1f3a249dc1f28395fb7dce98dfd0fe25885/pandas-1.1.0-cp37-cp37m-win_amd64.whl#sha256=a15835c8409d5edc50b4af93be3377b5dd3eb53517e7f785060df1f06f6da0e2:
        Expected sha256 a15835c8409d5edc50b4af93be3377b5dd3eb53517e7f785060df1f06f6da0e2
             Got        169763fb0142fa7955329adb9617e67b39cf8f1f3e67f2d01e4f539673cca41c



In [7]:
# imports 
import random

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    TextDataset, DataCollatorForLanguageModeling,
    Trainer, TrainingArguments)

ModuleNotFoundError: No module named 'pandas'

### Set up Transformer neural network

In [5]:
tokenizer = AutoTokenizer.from_pretrained('gpt2')
model = AutoModelForCausalLM.from_pretrained('gpt2')

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['h.0.attn.masked_bias', 'h.1.attn.masked_bias', 'h.2.attn.masked_bias', 'h.3.attn.masked_bias', 'h.4.attn.masked_bias', 'h.5.attn.masked_bias', 'h.6.attn.masked_bias', 'h.7.attn.masked_bias', 'h.8.attn.masked_bias', 'h.9.attn.masked_bias', 'h.10.attn.masked_bias', 'h.11.attn.masked_bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
block_size = tokenizer.max_len
train_dataset = TextDataset(tokenizer=tokenizer, file_path=f'data_{handle}_train.txt', block_size=block_size, overwrite_cache=True)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
epochs = 4  # limit before overfitting

NameError: name 'handle' is not defined

### Set up Training Args

In [ ]:
training_args = TrainingArguments(
    output_dir=f'output/{handle}',
    overwrite_output_dir=True,
    do_train=True,
    num_train_epochs=epochs,
    per_device_train_batch_size=1,
    logging_steps=5,
    save_steps=0,
    seed=random.randint(0,2**32-1))

### Set up Trainer

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    prediction_loss_only=True)

### Training neural network

In [ ]:
trainer.train()

In [4]:
def finetune():
    global trainer, tokenizer

    tokenizer = AutoTokenizer.from_pretrained('gpt2')
    model = AutoModelForCausalLM.from_pretrained('gpt2')

    block_size = tokenizer.max_len
    train_dataset = TextDataset(tokenizer=tokenizer, file_path=f'data_{handle}_train.txt', block_size=block_size, overwrite_cache=True)
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
    epochs = 4  # limit before overfitting

    training_args = TrainingArguments(
        output_dir=f'output/{handle}',
        overwrite_output_dir=True,
        do_train=True,
        num_train_epochs=epochs,
        per_device_train_batch_size=1,
        logging_steps=5,
        save_steps=0,
        seed=random.randint(0,2**32-1))

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
        prediction_loss_only=True)

    # Training neural network
    trainer.train()

predictions = []

def predict():
    # prepare input
    start_with_bos = '<|endoftext|> ' + start
    encoded_prompt = tokenizer(start_with_bos, add_special_tokens=False, return_tensors="pt").input_ids
    encoded_prompt = encoded_prompt.to(trainer.model.device)

    # prediction
    output_sequences = trainer.model.generate(
        input_ids=encoded_prompt,
        max_length=160,
        min_length=10,
        temperature=1.,
        top_p=0.95,
        do_sample=True,
        num_return_sequences=20
        )
    stop_token = '<|endoftext|>'
    generated_sequences = []

    # decode prediction
    for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
        generated_sequence = generated_sequence.tolist()
        text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)
        limit = text.find(stop_token, 1)
        text = text[: limit if limit != -1 else None]

        # we also remove new lines which happen due to pre-trained gpt-2 and twitter scraping
        limit = text.find('\n')
        text = text[: limit if limit != -1 else None]
        generated_sequence = text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)) :]
        generated_sequences.append(clean_prediction(generated_sequence))

    for i, g in enumerate(generated_sequences):
        predictions.append([start, ' '.join([start, g])])

    # log predictions
    print({'examples': wandb.Table(data=predictions, columns=['Input', 'Prediction'])})
